<a href="https://colab.research.google.com/github/jessalynlc/IDS705_final_project/blob/main/data_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
import json
import pandas as pd
import numpy as np

#DATA PULL

Data pull for all except wet bulb temperatures and Henry Hub are hourly and spans Jan 1 2018 - April 12, 2025. Henry Hub natural gas spot prices are available from 1997, but only data from 2018-2025 were used to align with the other datasets.

Load, LMP, fuel mix, temperature from https://www.gridstatus.io/datasets

Henry Hub Spot Prices from https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm

Wet bulb temperatures from https://www.planoweather.com/wxwetbulbdetail.php?

## Loading raw data files stored in github

In [2]:
#RT load data
load_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/ercot_load_by_forecast_zone.csv"

#RT LMP data
busavg_1_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUSAVG_LMP_Jan12018-Jan12019.csv"
busavg_2_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUSAVG_LMP_Jan12019-Jan12020.csv"
busavg_3_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUSAVG_LMP_Jan12020-Dec312020.csv"
busavg_4_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUS_AVG_LMP_Dec312020-Dec312021.csv"
busavg_5_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUS_AVG_LMP_Dec312021-Dec312022.csv"
busavg_6_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUS_AVG_LMP_Dec312022-Dec312023.csv"
#don't forget to delete Dec 30 data from here because it's in last data file!
busavg_7_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUS_AVG_LMP_Dec302023-Dec292024.csv"
busavg_8_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/HB_BUS_AVG_LMP_Dec292024_Apr132025.csv"

#fuel mix data
fuelmix_1_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Jan12018-Jan12019.csv"
fuelmix_2_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Jan12019-Jan12020.csv"
fuelmix_3_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Jan12020_Dec312020.csv"
fuelmix_4_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Dec312020_Dec312021.csv"
fuelmix_5_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Dec312021-Dec312022.csv"
fuelmix_6_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Dec312022-Dec312023.csv"
fuelmix_7_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Dec312023-Dec302024.csv"
fuelmix_8_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/fuel_mix_Dec302024-Apr132025.csv"

#temp data
temp_1_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Jan12018-Jan12019.csv"
temp_2_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Jan12019-Jan12020.csv"
temp_3_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Jan12020-Dec312020.csv"
temp_4_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Dec312020-Dec312021.csv"
temp_5_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Dec312021-Dec312022.csv"
temp_6_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Dec312022-Dec312023.csv"
temp_7_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Dec312023-Dec302024_v2.csv"
temp_8_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/main/Raw_Data/temp_Dec302024-Apr132025.csv"

#Henry Hub Data (daily) (Dollars per Million Btu)
henry_hub_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/refs/heads/main/Raw_Data/Henry_Hub_Natural_Gas_Spot_Price_updated.csv"

#Wet Bulb Temperature
wet_bulb_url = "https://raw.githubusercontent.com/jessalynlc/IDS705_final_project/refs/heads/main/Raw_Data/wet_bulb_temp.xlsx"

In [5]:
load = pd.read_csv(load_url)

busavg_1 = pd.read_csv(busavg_1_url)
busavg_2 = pd.read_csv(busavg_2_url)
busavg_3 = pd.read_csv(busavg_3_url)
busavg_4 = pd.read_csv(busavg_4_url)
busavg_5 = pd.read_csv(busavg_5_url)
busavg_6 = pd.read_csv(busavg_6_url)
busavg_7 = pd.read_csv(busavg_7_url)
busavg_8 = pd.read_csv(busavg_8_url)

fuelmix_1 = pd.read_csv(fuelmix_1_url)
fuelmix_2 = pd.read_csv(fuelmix_2_url)
fuelmix_3 = pd.read_csv(fuelmix_3_url)
fuelmix_4 = pd.read_csv(fuelmix_4_url)
fuelmix_5 = pd.read_csv(fuelmix_5_url)
fuelmix_6 = pd.read_csv(fuelmix_6_url)
fuelmix_7 = pd.read_csv(fuelmix_7_url)
fuelmix_8 = pd.read_csv(fuelmix_8_url)

temp_1 = pd.read_csv(temp_1_url)
temp_2 = pd.read_csv(temp_2_url)
temp_3 = pd.read_csv(temp_3_url)
temp_4 = pd.read_csv(temp_4_url)
temp_5 = pd.read_csv(temp_5_url)
temp_6 = pd.read_csv(temp_6_url)
temp_7 = pd.read_csv(temp_7_url)
temp_8 = pd.read_csv(temp_8_url)

henry_hub = pd.read_csv(henry_hub_url)
wet_bulb = pd.ExcelFile(wet_bulb_url)
sheet_names = wet_bulb.sheet_names

## Cleaning up Wet Bulb file

This is daily wet bulb temperature data from Plano, Texas. To align it with hourly data from GridStatus, we assume that the wet bulb temperature remains constant across all 24 hours of each day.

In [6]:
#Cleaning up wet_bulb file
all_data = []

for sheet in sheet_names:
    df = pd.read_excel(wet_bulb_url, sheet_name=sheet, skiprows=3)
    df = df.iloc[:, :13]  # Only keep 'Day' + Jan–Dec columns
    df = df.dropna(how='all')  # Drop empty rows

    # Melt the dataframe from wide to long format
    df_long = df.melt(id_vars='Day', var_name='month', value_name='wet_bulb')
    df_long['year'] = int(sheet)

    # Convert to datetime
    df_long['date'] = pd.to_datetime(df_long['year'].astype(str) + '-' + df_long['month'] + '-' + df_long['Day'].astype(str), errors='coerce')

    df_long = df_long.dropna(subset=['date'])  # Drop invalid dates (e.g., Feb 30)

    all_data.append(df_long[['date', 'wet_bulb']])

# Combine all years
wet_bulb_all = pd.concat(all_data, ignore_index=True).sort_values('date').reset_index(drop=True)


In [7]:
#Changing Henry Hub and Wet Bulb Temp data to hourly
wet_bulb_all['date'] = pd.to_datetime(wet_bulb_all['date'])
wet_bulb_all['hour'] = 0  # initialize an hour column

# Repeat each row 24 times (for 0–23 hours)
wet_bulb_hourly = wet_bulb_all.loc[wet_bulb_all.index.repeat(24)].reset_index(drop=True)
wet_bulb_hourly['hour'] = list(range(24)) * len(wet_bulb_all)

wet_bulb_hourly['interval_start_local'] = wet_bulb_hourly['date'] + pd.to_timedelta(wet_bulb_hourly['hour'], unit='h')
wet_bulb_hourly = wet_bulb_hourly[['interval_start_local', 'wet_bulb']]

## Cleaning up Henry Hub file

This dataset contains daily Henry Hub natural gas spot prices in `$/MMBTU`. To align with the hourly electricity data from GridStatus, we assume that the daily gas price remains constant across all 24 hours of each day as short-term intraday variation in spot gas prices is typically minimal for RT LMP forecasting.

Spot prices are not reported on weekends or federal holidays because the Henry Hub market (like many financial commodity markets) typically follows a weekday trading schedule, with no active settlement on non-business days. To address these gaps, we filled missing values using the nearest available gas price, an approach that helps maintain continuity while preserving the broader price signal.

In [8]:
#Convert to datetime and sort
henry_hub['date'] = pd.to_datetime(henry_hub['date'])
henry_hub = henry_hub.sort_values('date').reset_index(drop=True)

#Create a full daily date range
full_dates = pd.DataFrame({'date': pd.date_range(start=henry_hub['date'].min(),
                                                  end=henry_hub['date'].max(), freq='D')})

#Merge with original data
henry_hub_full = full_dates.merge(henry_hub, on='date', how='left')

#Fill missing prices using nearest value (forward-fill, then back-fill)
henry_hub_full['Henry Hub Natural Gas Spot Price Dollars per Million Btu'] = (
    henry_hub_full['Henry Hub Natural Gas Spot Price Dollars per Million Btu']
    .ffill()
    .bfill()
)

henry_hub_full = henry_hub_full.sort_values('date').reset_index(drop=True)

#Repeat each row 24 times (one for each hour)
henry_hub_hourly = henry_hub_full.loc[henry_hub_full.index.repeat(24)].reset_index(drop=True).copy()

#Create the hour column (0–23 for each day)
henry_hub_hourly['hour'] = list(range(24)) * len(henry_hub_full)

#Create the hourly timestamp
henry_hub_hourly['interval_start_local'] = henry_hub_hourly['date'] + pd.to_timedelta(henry_hub_hourly['hour'], unit='h')

#Select and rename columns if needed
henry_hub_hourly = henry_hub_hourly[['interval_start_local', 'Henry Hub Natural Gas Spot Price Dollars per Million Btu']]

<ipython-input-8-a9009abb53f3>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  henry_hub['date'] = pd.to_datetime(henry_hub['date'])


## Combining all csv files

In [9]:
#combining csv files

#cleaning this file because have an overlapping day
busavg_7_temp = busavg_7.drop(index=range(24))
busavg_lmp = pd.concat([busavg_1, busavg_2, busavg_3, busavg_4, busavg_5, busavg_6, busavg_7_temp, busavg_8])

fuel_mix = pd.concat([fuelmix_1, fuelmix_2, fuelmix_3, fuelmix_4, fuelmix_5, fuelmix_6, fuelmix_7, fuelmix_8])

temp = pd.concat([temp_1, temp_2, temp_3, temp_4, temp_5, temp_6, temp_7, temp_8])

In [10]:
#num of rows should match

#not really sure why load is missing data, so i'm just going to fill.NA later to simplify, could be worth looking into issue later
print(load.shape)
print(busavg_lmp.shape)
print(fuel_mix.shape)
print(temp.shape)

#load.to_csv('load.csv', index=False)

(63689, 9)
(63815, 6)
(63815, 12)
(63815, 14)


In [11]:
#dropping unnecessary columns
load = load.drop(columns=['interval_end_local', 'interval_start_local', 'interval_end_utc'])
busavg_lmp = busavg_lmp.drop(columns=['interval_end_local', 'interval_start_local', 'interval_end_utc', 'location'])
fuel_mix = fuel_mix.drop(columns=['interval_end_local', 'interval_start_local', 'interval_end_utc'])
temp = temp.drop(columns=['interval_end_local', 'interval_start_local', 'interval_end_utc', 'publish_time_local', 'publish_time_utc'])

#turning interval_start_local to datetime objects
for df in [load, busavg_lmp, fuel_mix, temp]:
    df['interval_start_utc'] = pd.to_datetime(df['interval_start_utc'], utc = True)


In [12]:
merged = load.merge(busavg_lmp, on='interval_start_utc', how='left')
merged = merged.merge(fuel_mix, on='interval_start_utc', how='left')
all_hourly_data = merged.merge(temp, on='interval_start_utc', how='left')

In [13]:
all_hourly_data['interval_start_local'] = all_hourly_data['interval_start_utc'].dt.tz_convert('America/Chicago')

# Extract date and hour
all_hourly_data['date'] = all_hourly_data['interval_start_local'].dt.date
all_hourly_data['hour'] = all_hourly_data['interval_start_local'].dt.hour

In [14]:
all_hourly_data = all_hourly_data.drop(columns=['interval_start_utc'])

In [15]:
cols = ['interval_start_local', 'date', 'hour'] + [col for col in all_hourly_data.columns if col not in ['interval_start_local', 'date', 'hour']]
all_hourly_data = all_hourly_data[cols]

In [16]:
# Convert 'interval_start_local' in wet_bulb_hourly and henry_hub_hourly to the same timezone as all_hourly_data
wet_bulb_hourly['interval_start_local'] = pd.to_datetime(wet_bulb_hourly['interval_start_local']).dt.tz_localize('UTC').dt.tz_convert('America/Chicago')
henry_hub_hourly['interval_start_local'] = pd.to_datetime(henry_hub_hourly['interval_start_local']).dt.tz_localize('UTC').dt.tz_convert('America/Chicago')


# Merge all three
merged_df = (
    all_hourly_data
    .merge(wet_bulb_hourly, on='interval_start_local', how='left')
    .merge(henry_hub_hourly, on='interval_start_local', how='left')
)

In [17]:
merged_df.to_csv('all_hourly_data.csv', index = False)